In [82]:
import torchvision
from torchvision.io.image import read_image
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights
from torchvision.models.detection import ssd300_vgg16, SSD300_VGG16_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
import torch
from torchvision import transforms

import cv2 as cv
from PIL import Image
import numpy as np

import glob

In [73]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Step 1: Initialize model with the best available weights
weights = SSD300_VGG16_Weights.DEFAULT
# model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
model = ssd300_vgg16(weights=weights)
# model.to('cuda')
model = model.eval()
print("")

Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to C:\Users\evan_/.cache\torch\hub\checkpoints\ssd300_vgg16_coco-b556d3b4.pth
100.0%


In [79]:
# img = torch.tensor(read_image("dataset/train/PXL_20201107_214721193_jpg.rf.fb52dba4cf2addec4a3d4cc2e9fa527b.jpg"), device=device)
# filepath = "dataset/train/PXL_20201107_214721193_jpg.rf.fb52dba4cf2addec4a3d4cc2e9fa527b.jpg"
# pic = Image.open(filepath)
# pic = pic.resize((320, 320))
# print(pic)
# pic.show()

# pic = pic*255
# print(pic)
# picTorch = transforms.ToTensor()(pic).unsqueeze(0)
# picTorch.resize((320, 320))
# print(type(pic))
# trans = SSDLite320_MobileNet_V3_Large_Weights.COCO_V1.transforms()
# img = trans(pic2)
# print(pic.shape)
# img = read_image("dataset/train/IMG_6818_jpg.rf.4cf2cbc673b8173c113825f52ed30a16.jpg")
img = read_image("elephant.jpg")

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()
# print(preprocess(pic).shape)

# Step 3: Apply inference preprocessing transforms
batch = [preprocess(img)]

# Step 4: Use the model and visualize the prediction
prediction = model(batch)[0]
labels = [weights.meta["categories"][i] for i in prediction["labels"]]

print(prediction["boxes"][:5])
print(labels[:5])

# get score for all the predicted objects
# pred_scores = prediction['scores'].detach().cpu().numpy()
# get all the predicted bounding boxes
# pred_bboxes = prediction['boxes'].detach().cpu().numpy()
# for i in range(len(prediction["scores"])):
#     if (prediction["scores"][i] > 0.9):
#         preds
# boxes = pred_bboxes[pred_scores >= 0.90].astype(np.int32)
# boxes = torch.tensor(boxes)
# labels = prediction['labels'][:len(boxes)]
scores = prediction['scores'][:5]
print(scores)

box = draw_bounding_boxes(img, boxes=prediction["boxes"][:5],
                          labels=labels[:5],
                          colors="red",
                          width=1, font_size=30)
im = to_pil_image(box.detach())
# im.show()


# image = cv.cvtColor(np.asarray(img), cv.COLOR_BGR2RGB)
# for i, box in enumerate(boxes):
#     color = COLORS[labels[i]]
#     cv.rectangle(
#         image,
#         (int(box[0]), int(box[1])),
#         (int(box[2]), int(box[3])),
#         color, 2
#     )
#     cv.putText(image, classes[i], (int(box[0]), int(box[1]-5)),
#                 cv.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, 
#                 lineType=cv.LINE_AA)


tensor([[ 335.2332,    0.0000, 1166.3341,  630.0000],
        [ 191.2635,  234.5848,  356.4376,  390.4071],
        [1013.8081,   84.4025, 1189.9844,  517.3238],
        [ 120.9551,  205.6234,  214.7919,  375.5619],
        [ 127.1045,   22.7642,  624.1195,  370.3398]],
       grad_fn=<SliceBackward0>)
['elephant', 'elephant', 'elephant', 'elephant', 'elephant']
tensor([0.9923, 0.6977, 0.0790, 0.0649, 0.0582], grad_fn=<SliceBackward0>)


In [115]:
def save_cropped_objects (filepath, num_imgs, id):
	img = read_image(filepath)
	preprocess = weights.transforms()
	batch = [preprocess(img)]
	prediction = model(batch)[0]
	labels = [weights.meta["categories"][i] for i in prediction["labels"]]
	im = to_pil_image(img)
	for i in range(num_imgs):
		coords = prediction["boxes"][i+1].tolist()
		# print(coords)
		im1 = im.crop(coords)
		im1.save("cropped_imgs/" + str(id) + "_" + str(i) + ".jpg")

In [113]:
save_cropped_objects("dataset/elephant.jpg", 5, 1)

[335.2332458496094, 0.0, 1166.3341064453125, 630.0]
[191.26345825195312, 234.5847625732422, 356.4375915527344, 390.4071350097656]
[1013.80810546875, 84.40252685546875, 1189.984375, 517.3237915039062]
[120.95512390136719, 205.6233673095703, 214.79188537597656, 375.5619201660156]
[127.10447692871094, 22.764230728149414, 624.1195068359375, 370.3397521972656]


In [114]:
filepath = 'dataset/raw/train/'
for i, filename in enumerate(glob.glob(filepath + '*.jpg')):
	save_cropped_objects(filename, 5, i)

filepath = 'dataset/raw/test/'
for i, filename in enumerate(glob.glob(filepath + '*.jpg')):
	save_cropped_objects(filename, 5, i)

filepath = 'dataset/raw/valid/'
for i, filename in enumerate(glob.glob(filepath + '*.jpg')):
	save_cropped_objects(filename, 5, i)

[1221.5404052734375, 1040.736572265625, 1536.0, 1969.666259765625]
[822.5318603515625, 1409.1788330078125, 950.5494995117188, 1576.4169921875]
[569.8259887695312, 1719.4132080078125, 976.0238037109375, 2029.0206298828125]
[1181.408447265625, 1478.49951171875, 1516.5653076171875, 2010.5194091796875]
[743.0154418945312, 1705.9644775390625, 1138.3199462890625, 2032.8861083984375]
[35.498592376708984, 1606.7186279296875, 978.201904296875, 2014.477294921875]
[283.37384033203125, 1627.5570068359375, 737.146728515625, 2008.6561279296875]
[958.663330078125, 1069.3819580078125, 1172.77587890625, 1312.0506591796875]
[525.2299194335938, 1599.8712158203125, 1374.254638671875, 2024.158203125]
[323.8945617675781, 1802.7266845703125, 691.83544921875, 2021.288818359375]
[1259.6219482421875, 1026.301025390625, 1536.0, 1968.029052734375]
[200.61553955078125, 1101.8226318359375, 714.263916015625, 1628.865966796875]
[1257.3668212890625, 1260.546875, 1499.72509765625, 1596.4954833984375]
[1127.190673828125